# Vulnerability and Housing Prices

In [208]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re

plt.style.use('seaborn-whitegrid')

In [49]:
"""County Demographics - taken from https://opendata.maryland.gov/Demographic/Choose-Maryland-Compare-Counties-Demographics/pa7d-u6hs"""
county_demographics = pd.read_csv("county_demographics.csv")
county_demographics.head()

,County,"Total Population, 2018","Total Population, 2010","Total Population, 2000","Population Change, 2000-2010",Population Density per Square Mile,Median Age,Per Capita Personal Income ($ Dollars),Median Household Income ($ Dollars),Total Personal Income ($ Thousands)
0,Baltimore City,602495,620961,651262,-30301,7671.5,35.6,49168,46762,30073681
1,Wicomico,103195,98733,84638,14095,263.7,35.9,40896,52341,4209176
2,St. Mary's,112664,105151,86196,18955,294.4,36.8,54921,82433,6187732
3,Somerset,25675,26470,24746,1724,82.8,37.4,31327,40320,811937
4,Prince George's,909308,863420,801091,62329,1788.8,37.5,47365,80858,43232981


In [50]:
"""Socioeconomic Characteristics per County - taken from https://opendata.maryland.gov/Demographic/Maryland-Jurisdictions-Socioeconomic-Characteristi/v67y-zu4n"""
county_socioeconomic = pd.read_csv("county_socioeconomic.csv")
county_socioeconomic.head()

,Jurisdictions,Total Households,Population 25 years and older,Less than 9th Grade,High School no Diploma,High School Diploma,Some College no degree,Associates degree,Bachelor's degree,Graduate or Professional,...,Male,Female,White Alone,Black Alone,Asian Alone,American Indian/Alaska Native Alone,Native Hawaiian/Pacific Islander Alone,Some Other Race Alone,Two or More Races,Hispanic or Latino (of any race)
0,Allegany County,28167,50793,1454,4089,21302,9999,4825,4733,4391,...,38077,34983,64695,6021,580,118,34,167,1445,1233
1,Anne Arundel County,204829,382657,8852,22100,93862,78474,28736,88157,62476,...,276842,282895,416331,89001,20103,1222,383,12470,20227,39402
2,Baltimore County,312826,569088,18308,33161,154724,111102,40101,121891,89801,...,391102,434564,519063,226879,47772,2110,394,7828,21620,40850
3,Calvert County,31479,60627,1065,3001,18704,15532,4570,10038,7717,...,44966,45561,73873,11124,1371,77,0,530,3552,3092
4,Caroline County,12010,22037,1291,2389,9207,4171,1608,2112,1259,...,15822,16831,26419,4411,187,30,16,623,967,2101


In [51]:
"""County Vulnerability - taken from https://svi.cdc.gov"""
county_vulnerability = pd.read_csv("county_vulnerability.csv")
county_vulnerability.head()

,FID,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,24,MARYLAND,MD,Anne Arundel,24003,"Anne Arundel County, Maryland",414.887094,559737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32248.0,1853.0,6.0,0.3,553075.0
1,1,24,MARYLAND,MD,Baltimore,24005,"Baltimore County, Maryland",598.361844,825666.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60233.0,2259.0,7.4,0.3,821710.0
2,2,24,MARYLAND,MD,Calvert,24009,"Calvert County, Maryland",213.197148,90527.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5327.0,744.0,6.0,0.8,67719.0
3,3,24,MARYLAND,MD,Carroll,24013,"Carroll County, Maryland",447.600142,167535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7249.0,711.0,4.4,0.4,137277.0
4,4,24,MARYLAND,MD,Cecil,24015,"Cecil County, Maryland",346.276658,102175.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7107.0,782.0,7.0,0.8,85648.0


In [52]:
"""New Construction - taken from https://opendata.maryland.gov/Housing/Maryland-Total-New-Parcels-For-Residential-Develop/6umw-84d2"""
county_development = pd.read_csv("county_development.csv")
county_development.head()

,Date created,Year,MARYLAND,Allegany County,Anne Arundel County,Baltimore City,Baltimore County,Calvert County,Caroline County,Carroll County,...,Kent County,Montgomery County,Prince George's County,Queen Anne's County,Somerset County,St. Mary's County,Talbot County,Washington County,Wicomico County,Worcester County
0,5/15/2017,2015,9712,16,1625,268,918,118,12,428,...,38,1186,1746,180,29,160,52,191,135,62
1,5/15/2017,2013,10346,32,1359,237,715,269,27,359,...,40,1624,1384,187,12,444,33,223,97,92
2,5/15/2017,2006,19814,137,1604,328,1813,393,321,654,...,149,1429,2920,376,167,868,287,1086,719,821
3,5/15/2017,2008,10380,112,1160,239,728,234,76,251,...,71,963,1473,205,93,624,172,356,282,328
4,5/15/2017,2012,8599,62,1171,84,648,264,25,290,...,22,1079,979,160,18,489,43,142,123,83


In [53]:
"""Find if vulnerability is correlated to less development"""

'Find if vulnerability is correlated to less development'

In [54]:
print(county_demographics.shape)
merge_name_demo = "County"

(24, 10)


In [55]:
print(county_socioeconomic.shape)
merge_name_socio = "Jurisdicions"

(24, 41)


In [56]:
print(county_vulnerability.shape)
merge_name_vuln = "COUNTY"

(24, 124)


In [127]:
"""Because data is from 2012 to 2016, the average data has been taken in this context"""

county_development = county_development.sort_values("Year")
county_2012_data = county_development.loc[(county_development["Year"] >= 2012)]
county_12_16_avg = county_2012_data.groupby("Date created").mean()
county_melt = county_12_16_avg.melt()

county_melt

#Regex out the vals for this

,variable,value
0,Year,2013.50
1,MARYLAND,9710.75
2,Allegany County,31.75
3,Anne Arundel County,1455.00
4,Baltimore City,185.25
5,Baltimore County,772.25
6,Calvert County,221.25
7,Caroline County,23.25
8,Carroll County,330.00
9,Cecil County,164.75


In [139]:
dc_development = pd.read_csv("dc_development.csv")
dc_residential = dc_development.loc[(dc_development["PROJECTTYPE"] == "RESIDENTIAL")]
print(dc_residential.info())

"""Count of DC residential productions is 60 in 2014. We're taking this to mean about 60 residential construction projects in 2014, given this context"""

DC_projects_dict = {"variable":["District of Columbia"], "value":["60.0"]}

dc_projects_data = pd.DataFrame.from_dict(DC_projects_dict)

dc_projects_data

"""Add the DC data to the residential parcels data."""

mass_projects_index = pd.concat([dc_projects_data, county_melt])

print(mass_projects_index)

mass_projects_index.to_csv("mass_projects_index.csv") # headed to get cleaned in openrefine.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 70 to 215
Data columns (total 13 columns):
OBJECTID            60 non-null int64
PROJECTNAME         60 non-null object
LOCATION            60 non-null object
WARD                60 non-null int64
DEVELOPER           60 non-null object
ARCHITECT           60 non-null object
STATUS              60 non-null object
UNITS               60 non-null float64
TYPE                60 non-null object
SQFT                0 non-null float64
ESTVALUEINMILION    54 non-null float64
ESTDELIVERY         60 non-null object
PROJECTTYPE         60 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 4.7+ KB
None
                  variable    value
0     District of Columbia     60.0
0                     Year   2013.5
1                 MARYLAND  9710.75
2          Allegany County    31.75
3      Anne Arundel County     1455
4           Baltimore City   185.25
5         Baltimore County   772.25
6           Calvert County   221.

In [170]:
"""Changed values to match that of main index later on"""

mass_project_index = pd.read_csv("mass_projects_index_cleaned.csv") #Cleaned out the counties

mass_project_index

,Unnamed: 0,variable,value
0,0,District of Columbia,60.00
1,1,MARYLAND,9710.75
2,2,Allegany,31.75
3,3,Anne Arundel,1455.00
4,4,Baltimore City,185.25
5,5,Baltimore,772.25
6,6,Calvert,221.25
7,7,Caroline,23.25
8,8,Carroll,330.00
9,9,Cecil,164.75


# Vulnerability Index

In [90]:
"""DC Vulnerability - taken from https://svi.cdc.gov"""
dc_vulnerability = pd.read_csv("dc_vulnerability.csv")

dc_vulnerability

,FID,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,11,DISTRICT OF COLUMBIA,DC,District of Columbia,11001,"District of Columbia, District of Columbia",61.14504,659009.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,33419.0,1818.0,5.2,0.3,1134734.0


In [60]:
"""Merging county vulnerability and DC vulnerability to compare"""
county_combined_vulnerability = pd.concat([dc_vulnerability, county_vulnerability])

In [61]:
county_combined_vulnerability.head()

,FID,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,0,11,DISTRICT OF COLUMBIA,DC,District of Columbia,11001,"District of Columbia, District of Columbia",61.145040,659009.0,0.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,33419.0,1818.0,5.2,0.3,1134734.0
0,0,24,MARYLAND,MD,Anne Arundel,24003,"Anne Arundel County, Maryland",414.887094,559737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32248.0,1853.0,6.0,0.3,553075.0
1,1,24,MARYLAND,MD,Baltimore,24005,"Baltimore County, Maryland",598.361844,825666.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60233.0,2259.0,7.4,0.3,821710.0
2,2,24,MARYLAND,MD,Calvert,24009,"Calvert County, Maryland",213.197148,90527.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5327.0,744.0,6.0,0.8,67719.0
3,3,24,MARYLAND,MD,Carroll,24013,"Carroll County, Maryland",447.600142,167535.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7249.0,711.0,4.4,0.4,137277.0


In [151]:
mass_index = county_combined_vulnerability[["COUNTY", "E_MUNIT", "E_CROWD", "E_NOVEH", "E_GROUPQ"]]
#Combine data of parcel data

In [152]:
mass_index

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ
0,District of Columbia,140304.0,9930.0,100656.0,39967.0
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0
1,Baltimore,62724.0,5144.0,24879.0,20798.0
2,Calvert,1447.0,389.0,1309.0,650.0
3,Carroll,3818.0,396.0,2687.0,3479.0
4,Cecil,2477.0,583.0,1733.0,1550.0
5,Frederick,10028.0,1354.0,4445.0,4624.0
6,Harford,10081.0,985.0,4552.0,1951.0
7,Queen Anne's,380.0,253.0,535.0,464.0
8,Washington,5428.0,1065.0,4594.0,8858.0


In [153]:
mass_index = mass_index.merge(county_demographics[["County", "Total Population, 2018", "Population Density per Square Mile","Median Household Income ($ Dollars)"]], left_on = "COUNTY", right_on = "County", how = "inner" )

In [154]:
mass_index.head()

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars)
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0,Anne Arundel,576031,1295.9,96133
1,Baltimore,62724.0,5144.0,24879.0,20798.0,Baltimore,828431,1345.5,73309
2,Calvert,1447.0,389.0,1309.0,650.0,Calvert,92003,416.3,100000
3,Carroll,3818.0,396.0,2687.0,3479.0,Carroll,168429,373.4,92890
4,Cecil,2477.0,583.0,1733.0,1550.0,Cecil,102826,292.0,72259


In [155]:
#mass_index = county_combined_vulnerability[["COUNTY", "E_MUNIT", "E_CROWD", "E_NOVEH", "E_GROUPQ"]]
#Combine data of parcel data

In [156]:
mass_index

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars)
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0,Anne Arundel,576031,1295.9,96133
1,Baltimore,62724.0,5144.0,24879.0,20798.0,Baltimore,828431,1345.5,73309
2,Calvert,1447.0,389.0,1309.0,650.0,Calvert,92003,416.3,100000
3,Carroll,3818.0,396.0,2687.0,3479.0,Carroll,168429,373.4,92890
4,Cecil,2477.0,583.0,1733.0,1550.0,Cecil,102826,292.0,72259
5,Frederick,10028.0,1354.0,4445.0,4624.0,Frederick,255648,353.5,91727
6,Harford,10081.0,985.0,4552.0,1951.0,Harford,253956,560.1,80922
7,Queen Anne's,380.0,253.0,535.0,464.0,Queen Anne's,50251,128.5,90696
8,Washington,5428.0,1065.0,4594.0,8858.0,Washington,150926,322.1,59828
9,Garrett,619.0,92.0,886.0,596.0,Garrett,29163,46.5,46899


In [157]:
"""Building up the DC row based on current data"""

DC_row = dc_vulnerability[["COUNTY", "E_MUNIT", "E_CROWD", "E_NOVEH", "E_GROUPQ"]]

DC_row.insert(5, "County", "District of Columbia")
DC_row.insert(6, "Total Population, 2018", 702445)
DC_row.insert(7, "Population Density per Square Mile", 11506.0)
DC_row.insert(8, "Median Household Income ($ Dollars)", 77649)

"""From Wikipedia.org: information about DC

Population Density: 11506.0
Total Population (2018): 702445
Median Household Income: 77649

"""

'From Wikipedia.org: information about DC\n\nPopulation Density: 11506.0\nTotal Population (2018): 702445\nMedian Household Income: 77649\n\n'

In [158]:
DC_row

,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars)
0,District of Columbia,140304.0,9930.0,100656.0,39967.0,District of Columbia,702445,11506.0,77649


In [159]:
mass_index = pd.concat([mass_index, DC_row])

In [160]:
mass_index = mass_index.reset_index()

In [161]:
mass_index.tail()

,index,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars)
20,20,Allegany,1835.0,254.0,3126.0,7644.0,Allegany,70975,177.0,42564
21,21,Prince George's,75968.0,11812.0,27760.0,18893.0,Prince George's,909308,1788.8,80858
22,22,Baltimore City,48374.0,4969.0,71257.0,24258.0,Baltimore City,602495,7671.5,46762
23,23,Somerset,767.0,80.0,1030.0,6572.0,Somerset,25675,82.8,40320
24,0,District of Columbia,140304.0,9930.0,100656.0,39967.0,District of Columbia,702445,11506.0,77649


In [162]:
mass_index.head()

,index,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars)
0,0,Anne Arundel,28410.0,3264.0,8089.0,16092.0,Anne Arundel,576031,1295.9,96133
1,1,Baltimore,62724.0,5144.0,24879.0,20798.0,Baltimore,828431,1345.5,73309
2,2,Calvert,1447.0,389.0,1309.0,650.0,Calvert,92003,416.3,100000
3,3,Carroll,3818.0,396.0,2687.0,3479.0,Carroll,168429,373.4,92890
4,4,Cecil,2477.0,583.0,1733.0,1550.0,Cecil,102826,292.0,72259


In [171]:
mass_index_changed = mass_index.merge(mass_project_index, left_on = "COUNTY", right_on ="variable",)

#Value will be the number of new residential parcels for development.

In [172]:
mass_index_changed

,index,COUNTY,E_MUNIT,E_CROWD,E_NOVEH,E_GROUPQ,County,"Total Population, 2018",Population Density per Square Mile,Median Household Income ($ Dollars),Unnamed: 0,variable,value
0,0,Anne Arundel,28410.0,3264.0,8089.0,16092.0,Anne Arundel,576031,1295.9,96133,3,Anne Arundel,1455.00
1,1,Baltimore,62724.0,5144.0,24879.0,20798.0,Baltimore,828431,1345.5,73309,5,Baltimore,772.25
2,2,Calvert,1447.0,389.0,1309.0,650.0,Calvert,92003,416.3,100000,6,Calvert,221.25
3,3,Carroll,3818.0,396.0,2687.0,3479.0,Carroll,168429,373.4,92890,8,Carroll,330.00
4,4,Cecil,2477.0,583.0,1733.0,1550.0,Cecil,102826,292.0,72259,9,Cecil,164.75
5,5,Frederick,10028.0,1354.0,4445.0,4624.0,Frederick,255648,353.5,91727,12,Frederick,555.00
6,6,Harford,10081.0,985.0,4552.0,1951.0,Harford,253956,560.1,80922,14,Harford,446.00
7,7,Queen Anne's,380.0,253.0,535.0,464.0,Queen Anne's,50251,128.5,90696,19,Queen Anne's,181.00
8,8,Washington,5428.0,1065.0,4594.0,8858.0,Washington,150926,322.1,59828,23,Washington,191.50
9,9,Garrett,619.0,92.0,886.0,596.0,Garrett,29163,46.5,46899,13,Garrett,70.00


In [173]:
index = mass_index_changed

In [190]:
index_cols = list(index.columns)

print(index_cols)

index.drop(['index','County', 'variable'], axis = 1, inplace = True)

['COUNTY', 'E_MUNIT', 'E_CROWD', 'E_NOVEH', 'E_GROUPQ', 'Total Population, 2018', 'Population Density per Square Mile', 'Median Household Income ($ Dollars)', 'Unnamed: 0', 'value']


KeyError: "['index' 'County' 'variable'] not found in axis"

In [192]:
index_backup = index

In [198]:
index_backup = index_backup.drop(index_backup.columns[8], axis=1)

In [200]:
"""Renaming columns for easier access and clarity"""

index_backup.columns = ["County", "large_Unt", "Crowded_Unt", "No_Vehicle", "Group_Unt", "Pop_Total", "Pop_Dens", "Median_inc", "Res_Parcel"]

In [203]:
"""Final Variable tables are taking shape"""

index = index_backup

index

,County,large_Unt,Crowded_Unt,No_Vehicle,Group_Unt,Pop_Total,Pop_Dens,Median_inc,Res_Parcel
0,Anne Arundel,28410.0,3264.0,8089.0,16092.0,576031,1295.9,96133,1455.00
1,Baltimore,62724.0,5144.0,24879.0,20798.0,828431,1345.5,73309,772.25
2,Calvert,1447.0,389.0,1309.0,650.0,92003,416.3,100000,221.25
3,Carroll,3818.0,396.0,2687.0,3479.0,168429,373.4,92890,330.00
4,Cecil,2477.0,583.0,1733.0,1550.0,102826,292.0,72259,164.75
5,Frederick,10028.0,1354.0,4445.0,4624.0,255648,353.5,91727,555.00
6,Harford,10081.0,985.0,4552.0,1951.0,253956,560.1,80922,446.00
7,Queen Anne's,380.0,253.0,535.0,464.0,50251,128.5,90696,181.00
8,Washington,5428.0,1065.0,4594.0,8858.0,150926,322.1,59828,191.50
9,Garrett,619.0,92.0,886.0,596.0,29163,46.5,46899,70.00


In [210]:
plt.scatter(index["Large_Unt"], ["Crowded_Unt"])

AttributeError: module 'matplotlib' has no attribute 'scatter'